In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
import tensorflow as tf


In [167]:
df = pd.read_csv('sorted_feature_matrix.csv')

In [168]:
df[df['Tickers']=="AAL"]

,Unnamed: 0,Date,Dividend,Volume,stock_price,fed_funds_rate,GDP,Tickers,debt_to_equity,EPS,return_on_equity,quick ratio,operating_ratio,inventory_turnover,pos_ma,neu_ma,neg_ma
0,69,2018-01-31,0.0,5.315047e+06,51.919825,1.41,4500.182000,AAL,-134.253165,0.807713,-0.941772,0.484585,0.476713,1.930838,0.285655,0.519859,0.194486
447,70,2018-02-28,0.1,5.315047e+06,51.919825,1.42,4549.064667,AAL,-134.253165,0.807713,-0.941772,0.484585,0.476713,1.930838,0.320593,0.266028,0.413378
905,71,2018-03-31,0.0,5.558910e+06,49.781805,1.51,4597.947333,AAL,-134.253165,0.807713,-0.941772,0.484585,0.476713,1.930838,0.800789,0.142719,0.056492
1371,72,2018-04-30,0.0,5.802773e+06,47.643785,1.69,4646.830000,AAL,-134.253165,0.807713,-0.941772,0.484585,0.476713,1.930838,0.563970,0.386575,0.049455
1837,73,2018-05-31,0.1,6.371183e+06,42.626889,1.70,4661.188333,AAL,-134.253165,0.807713,-0.941772,0.484585,0.476713,1.930838,0.029625,0.874360,0.096015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30519,133,2023-05-31,0.0,2.599978e+07,13.983333,5.06,5053.623333,AAL,-14.844027,1.370834,-0.184719,0.644943,0.496469,1.729324,0.000142,0.999418,0.000440
31001,134,2023-06-30,0.0,2.599978e+07,13.983333,5.08,5053.623333,AAL,-16.338655,2.048105,-0.305131,0.655535,0.466129,1.679386,0.351945,0.197356,0.450699
31483,135,2023-07-31,0.0,2.599978e+07,13.983333,5.08,5053.623333,AAL,-16.338655,2.048105,-0.305131,0.655535,0.466129,1.679386,0.444775,0.553510,0.001715
31964,136,2023-08-31,0.0,2.599978e+07,13.983333,5.08,5053.623333,AAL,-16.338655,2.048105,-0.305131,0.655535,0.466129,1.679386,0.239710,0.502044,0.258246


In [74]:
df = df[['Unnamed: 0', 'date', 'symbol', 'Dividend', 'volume', 'fed_funds_rate',
#        'gdp', 'debt_to_equity', 'eps', 'return_on_equity', 'quick_ratio',
       'operating_ratio', 'inventory_turnover', 'sa_neu','sa_pos', 'sa_neg', 'stock_price']]

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Date                1000 non-null   object 
 2   Dividend            1000 non-null   float64
 3   Volume              1000 non-null   float64
 4   stock_price         1000 non-null   float64
 5   fed_funds_rate      1000 non-null   float64
 6   GDP                 1000 non-null   float64
 7   Tickers             1000 non-null   object 
 8   debt_to_equity      1000 non-null   float64
 9   EPS                 1000 non-null   float64
 10  return_on_equity    1000 non-null   float64
 11  quick ratio         1000 non-null   float64
 12  operating_ratio     1000 non-null   float64
 13  inventory_turnover  1000 non-null   float64
 14  pos_ma              1000 non-null   float64
 15  neu_ma              1000 non-null   float64
 16  neg_ma 

In [86]:
# data = data.drop(columns=['Unnamed: 0']).set_index(['date', 'symbol', 'stock_price'])

In [136]:
tickers = df['Tickers'].unique().tolist()
list_of_df = []

for ticker in tickers:
    indiv_df = df[df['Tickers'] == ticker].copy()
    indiv_df['Ticker_Name'] = ticker
    list_of_df.append(indiv_df)

In [137]:
list_of_df

[     Unnamed: 0        Date  Dividend        Volume  stock_price  \
 0            69  2018-01-31       0.0  5.315047e+06    51.919825   
 447          70  2018-02-28       0.1  5.315047e+06    51.919825   
 905          71  2018-03-31       0.0  5.558910e+06    49.781805   
 
      fed_funds_rate          GDP Tickers  debt_to_equity       EPS  \
 0              1.41  4500.182000     AAL     -134.253165  0.807713   
 447            1.42  4549.064667     AAL     -134.253165  0.807713   
 905            1.51  4597.947333     AAL     -134.253165  0.807713   
 
      return_on_equity  quick ratio  operating_ratio  inventory_turnover  \
 0           -0.941772     0.484585         0.476713            1.930838   
 447         -0.941772     0.484585         0.476713            1.930838   
 905         -0.941772     0.484585         0.476713            1.930838   
 
        pos_ma    neu_ma    neg_ma Ticker_Name  
 0    0.285655  0.519859  0.194486         AAL  
 447  0.320593  0.266028  0.4133

In [182]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'symbol' column
df['Tickers'] = label_encoder.fit_transform(df['Tickers'])

# # Drop the original 'symbol' column if you no longer need it
# df.drop(columns=['symbol'], inplace=True)

# Define the number of time steps (lookback) and the number of features
lookback = 10  # You can adjust this based on your dataset
n_features = 10  # Update this to match the number of columns in your DataFrame

# # Create sequences of data for training
# def create_sequences(data, lookback):
#     X, y = [], []
#     for i in range(len(data) - lookback):
#         # Convert the DataFrame to a NumPy array using .values
#         X.append(data.iloc[i:i+lookback].values)  # Exclude 'date' and 'stock_price' columns
#         y.append(data.iloc[i+lookback])
#     return np.array(X), np.array(y)

tickers = df['Tickers'].unique().tolist()
list_of_df = []

for ticker in tickers:
    indiv_df = df[df['Tickers'] == ticker].copy()
    indiv_df['Ticker_Name'] = ticker
    list_of_df.append(indiv_df)
    
# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
scaled_data = []

# for df in list_of_df:
for indiv_df in list_of_df:
    if indiv_df['Ticker_Name'].values == "AAL":
        # Exclude 'date' & 'stock_price' columns
        numeric_columns = indiv_df.columns.difference(['Date', 'stock_price'])
        # Apply Min-Max scaling to the selected numeric columns
        indiv_df[numeric_columns] = scaler.fit_transform(indiv_df[numeric_columns])
        # Append the scaled DataFrame to scaled_data
        scaled_data.append(indiv_df)
print(indiv_df)
# # Create sequences of data
# # X, y = create_sequences(scaled_data, lookback)

# Assuming that your target variable is 'stock_price'
X = [df.drop(columns=['Date', 'stock_price']).values for df in scaled_data]
y = [df['stock_price'].values for df in scaled_data]

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=20)

from sklearn.model_selection import LeaveOneOut
k = 3  # Number of folds (you can adjust this)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Initialize a list to store the MSE for each fold
mse_scores = []

for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Train your model on X_train and y_train
    input_layer = Input(shape=(lookback, n_features), name='input_features')
    lstm_layer = LSTM(units=50, return_sequences=True)(input_layer)
    dropout_layer = Dropout(0.2)(lstm_layer)
    lstm_layer2 = LSTM(units=50, return_sequences=True)(dropout_layer)
    dropout_layer2 = Dropout(0.2)(lstm_layer2)
    lstm_layer3 = LSTM(units=50)(dropout_layer2)
    dropout_layer3 = Dropout(0.2)(lstm_layer3)
    output_layer = Dense(units=1)(dropout_layer3)

    model = Model(inputs=input_layer, outputs=output_layer)
    # Make predictions on the test set
    y_pred = model.predict(X_test)  # Replace 'model' with your model
    
    # Store the predictions and ground truth
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)

# Calculate the mean squared error (MSE) across all iterations
average_mse = np.mean(mse_scores)
print("Mean Squared Error:", average_mse)

print(X_train.shape)
print(X_test.shape)
# # X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
# # y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
# # X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
# # y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)
# # (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Build the LSTM model with time-aware features
# model = Sequential()
# model.add(layers.LSTM(units=10, activation='tanh', input_shape=(10,15)))
# model.add(layers.Dense(1, activation="linear"))

# input_layer = Input(shape=(lookback, n_features), name='input_features')
# lstm_layer = LSTM(units=50, return_sequences=True)(input_layer)
# dropout_layer = Dropout(0.2)(lstm_layer)
# lstm_layer2 = LSTM(units=50, return_sequences=True)(dropout_layer)
# dropout_layer2 = Dropout(0.2)(lstm_layer2)
# lstm_layer3 = LSTM(units=50)(dropout_layer2)
# dropout_layer3 = Dropout(0.2)(lstm_layer3)
# output_layer = Dense(units=1)(dropout_layer3)

# model = Model(inputs=input_layer, outputs=output_layer)

# # Alternative: Build the LSTM model
# # model = Sequential()
# # model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], len(features))))
# # model.add(LSTM(units=50, return_sequences=False))
# # model.add(Dense(units=25))
# # model.add(Dense(units=1))
# #----#

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model
# model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

# # Evaluate the model
# train_loss = model.evaluate(X_train, y_train, verbose=0)
# test_loss = model.evaluate(X_test, y_test, verbose=0)

# print(f'Training Loss: {train_loss}')
# print(f'Test Loss: {test_loss}')

# # Make predictions
# predictions = model.predict(X_test)

# # Inverse transform the predictions to get the actual stock prices
# predictions = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], predictions), axis=1))[:, -1]

# # Print or use predictions as needed
# train_predictions = model.predict(X_train)
# test_predictions = model.predict(X_test)

# # Inverse transform the predictions to the original scale
# train_predictions = scaler.inverse_transform(train_predictions)
# test_predictions = scaler.inverse_transform(test_predictions)

# # Plot the predictions
# plt.figure(figsize=(12, 6))
# plt.plot(data.index[sequence_length:len(X_train) + sequence_length], y_train, label='Actual Train Prices', color='blue')
# plt.plot(data.index[len(X_train) + sequence_length:], y_test, label='Actual Test Prices', color='green')
# plt.plot(data.index[sequence_length:len(X_train) + sequence_length], train_predictions, label='Predicted Train Prices', color='red')
# plt.plot(data.index[len(X_train) + sequence_length:], test_predictions, label='Predicted Test Prices', color='orange')
# plt.legend()
# plt.xlabel('Date')
# plt.ylabel('Stock Price')
# plt.title('Stock Price Prediction')
# plt.show()


       Unnamed: 0        Date  Dividend        Volume  stock_price  \
24440       13501  2022-04-30     0.000  4.078427e+06    62.177978   
24922       13502  2022-05-31     0.000  4.078427e+06    62.177978   
25404       13503  2022-06-30     0.000  4.078427e+06    62.177978   
25886       13504  2022-07-31     0.000  4.078427e+06    62.177978   
26368       13505  2022-08-31     0.000  4.078427e+06    62.177978   
26850       13506  2022-09-30     0.000  4.078427e+06    62.177978   
27332       13507  2022-10-31     0.000  4.078427e+06    62.177978   
27814       13508  2022-11-30     0.000  4.078427e+06    62.177978   
28296       13509  2022-12-31     0.000  4.078427e+06    62.177978   
28778       13510  2023-01-31     0.000  4.078427e+06    62.177978   
29260       13511  2023-02-28     0.000  2.597623e+06    70.367206   
29742       13512  2023-03-31     0.000  2.105297e+06    76.146563   
30224       13513  2023-04-30     0.015  2.148888e+06    77.826936   
30706       13514  2

/var/folders/n4/35k7kmsn57n5yqhz89vtwcfr0000gn/T/ipykernel_90712/1594089693.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if indiv_df['Ticker_Name'].values == "AAL":


ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=0.